In [11]:
!pip3 install pyspark


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("KaggleParticipation").config(
    "spark.driver.memory", "4g"
).config("spark.executor.memory", "4g").getOrCreate()


25/05/19 08:37:44 WARN Utils: Your hostname, LT-W-7826.local resolves to a loopback address: 127.0.0.1; using 192.168.31.254 instead (on interface en0)
25/05/19 08:37:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/19 08:37:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
users = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/users.csv",
    header=True,
    inferSchema=True,
)
competitions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv",
    header=True,
    inferSchema=True,
)
teams = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/teams.csv",
    header=True,
    inferSchema=True,
)
submissions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/submissions.csv",
    header=True,
    inferSchema=True,
)


In [5]:
for df, name in [
    (users, "users"),
    (competitions, "competitions"),
    (teams, "teams"),
    (submissions, "submissions"),
]:
    print(f"—— schema of {name} ——")
    df.printSchema()


—— schema of users ——
root
 |-- Id: integer (nullable = true)
 |-- UserName: string (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- RegisterDate: string (nullable = true)
 |-- PerformanceTier: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationSharingOptOut: string (nullable = true)

—— schema of competitions ——
root
 |-- Id: string (nullable = true)
 |-- Slug: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- HostSegmentTitle: string (nullable = true)
 |-- ForumId: string (nullable = true)
 |-- OrganizationId: string (nullable = true)
 |-- EnabledDate: string (nullable = true)
 |-- DeadlineDate: string (nullable = true)
 |-- ProhibitNewEntrantsDeadlineDate: string (nullable = true)
 |-- TeamMergerDeadlineDate: string (nullable = true)
 |-- TeamModelDeadlineDate: string (nullable = true)
 |-- ModelSubmissionDeadlineDate: string (nullable = true)
 |-- FinalLeaderboardHasBeenVerified: str

In [ ]:
submissions_df = submissions.alias("s")
teams_df = teams.alias("t")
users_df = users.alias("u")


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, to_date, row_number
from pyspark.sql.window import Window

compet_schema = StructType(
    [
        StructField("Id", StringType(), True),
        StructField("Slug", StringType(), True),
        StructField("DeadlineDate", StringType(), True),
    ]
)

raw_comp = (
    spark.read.option("header", True)
    .option("multiLine", True)
    .option("mode", "DROPMALFORMED")
    .schema(compet_schema)
    .csv(
        "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv"
    )
)

clean_comp = (
    raw_comp.filter(col("Id").rlike("^[0-9]+$"))
    .withColumn("CompetitionId", col("Id").cast(IntegerType()))
    .withColumn("DeadlineDate", to_date("DeadlineDate", "yyyy-MM-dd"))
    .select("CompetitionId", "Slug", "DeadlineDate")
    .alias("c")
)

clean_comp.show(5, truncate=False)


25/05/18 23:41:02 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 48, schema size: 3
CSV file: file:///Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv


+-------------+----+------------+
|CompetitionId|Slug|DeadlineDate|
+-------------+----+------------+
|1            |0.5 |NULL        |
|2            |0.1 |NULL        |
|3            |0.01|NULL        |
|4            |0.9 |NULL        |
|5            |0.75|NULL        |
+-------------+----+------------+
only showing top 5 rows



In [9]:
sub_with_team = submissions.alias("s").join(
    teams.alias("t"), col("s.TeamId") == col("t.Id"), "left"
)

sub_with_team_comp = sub_with_team.join(
    clean_comp.alias("c"), col("t.CompetitionId") == col("c.CompetitionId"), "left"
)

sub_with_user = sub_with_team_comp.join(
    users.alias("u"), col("s.SubmittedUserId") == col("u.Id"), "left"
)

sub_with_user.select(
    col("s.Id").alias("SubmissionId"),
    col("u.UserName"),
    col("t.TeamName"),
    col("c.Slug"),
    col("s.SubmissionDate"),
).show(5, truncate=False)


25/05/18 23:41:17 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 48, schema size: 3
CSV file: file:///Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv


+------------+----------+----------+----+--------------+
|SubmissionId|UserName  |TeamName  |Slug|SubmissionDate|
+------------+----------+----------+----+--------------+
|2249        |niall786  |nfsteam   |NULL|05/11/2010    |
|2250        |niall786  |nfsteam   |NULL|05/11/2010    |
|2253        |niall786  |nfsteam   |NULL|05/11/2010    |
|8479597     |yydsb2000 |huxiaoqing|NULL|09/22/2018    |
|3930970     |adriguille|AdriGuille|NULL|12/19/2016    |
+------------+----------+----------+----+--------------+
only showing top 5 rows



In [8]:
competition_dim = (
    clean_comp.dropDuplicates(["CompetitionId"])
    .withColumn("CompetitionKey", row_number().over(Window.orderBy("CompetitionId")))
    .alias("cd")
)

competition_dim.printSchema()
competition_dim.show(5, truncate=False)


root
 |-- CompetitionId: integer (nullable = true)
 |-- Slug: string (nullable = true)
 |-- DeadlineDate: date (nullable = true)
 |-- CompetitionKey: integer (nullable = false)



25/05/18 23:41:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:04 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 48, schema size: 3
CSV file: file:///Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv
25/05/18 23:41:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:05 WARN WindowExec: No Partition Defined for Window operation! Movin

+-------------+----+------------+--------------+
|CompetitionId|Slug|DeadlineDate|CompetitionKey|
+-------------+----+------------+--------------+
|1            |0.5 |NULL        |1             |
|2            |0.1 |NULL        |2             |
|3            |0.01|NULL        |3             |
|4            |0.9 |NULL        |4             |
|5            |0.75|NULL        |5             |
+-------------+----+------------+--------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import dayofmonth, month, year, hour, date_format

user_dim = (
    users.select("Id", "UserName", "Country", "PerformanceTier")
    .dropDuplicates()
    .withColumn("UserKey", row_number().over(Window.orderBy("Id")))
    .alias("ud")
)
user_dim.show(5, truncate=False)

team_dim = (
    teams.select("Id", "TeamName", "IsBenchmark")
    .dropDuplicates()
    .withColumn("TeamKey", row_number().over(Window.orderBy("Id")))
    .alias("td")
)
team_dim.show(5, truncate=False)

time_dim = (
    submissions.alias("s")
    .select("SubmissionDate")
    .dropDuplicates()
    .withColumn("TimeKey", to_date("SubmissionDate", "MM/dd/yyyy"))
    .filter(col("TimeKey").isNotNull())
    .withColumn("Day", dayofmonth("TimeKey"))
    .withColumn("Month", month("TimeKey"))
    .withColumn("Year", year("TimeKey"))
    .withColumn("Hour", hour("SubmissionDate"))
    .withColumn("DayOfWeek", date_format("TimeKey", "EEEE"))
    .select("TimeKey", "Day", "Month", "Year", "Hour", "DayOfWeek")
    .alias("tdim")
)
time_dim.show(5, truncate=False)


25/05/18 23:41:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:41:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/18 23:41:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/18 23:41:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/18 23:41:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/18 23:41:46 WARN RowBasedKeyVal

+---+------------+-------------+---------------+-------+
|Id |UserName    |Country      |PerformanceTier|UserKey|
+---+------------+-------------+---------------+-------+
|1  |kaggleteam  |NULL         |5              |1      |
|368|antgoldbloom|United States|2              |2      |
|381|iguyon      |United States|2              |3      |
|383|davidstephan|Australia    |0              |4      |
|384|gabewarren  |Australia    |0              |5      |
+---+------------+-------------+---------------+-------+
only showing top 5 rows



25/05/18 23:42:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------------+-----------+-------+
|Id |TeamName    |IsBenchmark|TeamKey|
+---+------------+-----------+-------+
|496|team1       |False      |1      |
|497|jonp        |False      |2      |
|499|Bwaas       |False      |3      |
|500|Thylacoleo  |False      |4      |
|501|pjonesdotcda|False      |5      |
+---+------------+-----------+-------+
only showing top 5 rows



+----------+---+-----+----+----+---------+
|TimeKey   |Day|Month|Year|Hour|DayOfWeek|
+----------+---+-----+----+----+---------+
|2011-03-02|2  |3    |2011|NULL|Wednesday|
|2012-01-03|3  |1    |2012|NULL|Tuesday  |
|2012-05-19|19 |5    |2012|NULL|Saturday |
|2012-12-01|1  |12   |2012|NULL|Saturday |
|2013-11-17|17 |11   |2013|NULL|Sunday   |
+----------+---+-----+----+----+---------+
only showing top 5 rows



In [11]:
facts = (
    sub_with_user.join(user_dim, col("s.SubmittedUserId") == col("ud.Id"), "left")
    .join(team_dim, col("s.TeamId") == col("td.Id"), "left")
    .join(competition_dim, col("t.CompetitionId") == col("cd.CompetitionId"), "left")
    .join(
        time_dim,
        to_date(col("s.SubmissionDate"), "MM/dd/yyyy") == col("tdim.TimeKey"),
        "left",
    )
    .select(
        col("s.Id").alias("SubmissionId"),
        col("tdim.TimeKey"),
        col("ud.UserKey"),
        col("td.TeamKey"),
        col("cd.CompetitionKey"),
        col("s.PublicScoreLeaderboardDisplay"),
        col("s.PrivateScoreLeaderboardDisplay"),
        col("s.IsAfterDeadline"),
    )
    .repartition(10)
)


In [ ]:
user_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/UserDim"
)
team_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TeamDim"
)
competition_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/CompetitionDim"
)
time_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TimeDim"
)
facts.write.mode("overwrite").parquet(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/SubmissionFact"
)


25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 23:42:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/18 2

In [ ]:
df = spark.read.parquet(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/SubmissionFact/part-00000-8494f44c-22e4-4a03-94ab-b3eb5d4b9eca-c000.snappy.parquet"
)
df.show()
df.printSchema()


+------------+----------+--------+-------+--------------+-----------------------------+------------------------------+---------------+
|SubmissionId|   TimeKey| UserKey|TeamKey|CompetitionKey|PublicScoreLeaderboardDisplay|PrivateScoreLeaderboardDisplay|IsAfterDeadline|
+------------+----------+--------+-------+--------------+-----------------------------+------------------------------+---------------+
|     6175027|2017-12-17|    NULL| 727468|          NULL|                      0.68876|                       0.69047|          false|
|     1924775|2015-09-03|   26632| 128680|          NULL|                      0.79616|                       0.79017|          false|
|    10546321|2019-03-20| 1007070|1810144|          NULL|                      0.86993|                       0.86498|          false|
|     5610561|2017-10-22|  601759| 572799|          NULL|                       0.2835|                        0.2842|          false|
|    37059017|2024-03-25|14961887|7143082|          NUL

25/05/19 07:30:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 908595 ms exceeds timeout 120000 ms
25/05/19 07:30:50 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/19 07:30:51 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [2]:
user_dim = spark.read.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/UserDim"
)

team_dim = spark.read.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TeamDim"
)

competition_dim = spark.read.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/CompetitionDim"
)

time_dim = spark.read.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TimeDim"
)

facts = spark.read.parquet(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/SubmissionFact"
)


## User Dim
Metric: Average Public Score per User

In [3]:
import pyspark.sql.functions as F

agg_user = facts.groupBy("UserKey").agg(
    F.avg("PublicScoreLeaderboardDisplay").alias("AvgPublicScore")
)
agg_user.show()


+--------+--------------------+
| UserKey|      AvgPublicScore|
+--------+--------------------+
| 6439767|     3994.4158366932|
| 5442355|  0.7885235802469136|
|  750411|   156717.6785662082|
| 1426612|  12.276170952380953|
| 1634073|  0.3836131111111112|
|  282092| 4.942466592068354E8|
| 1403156|  15.027818469387757|
| 1589657|  0.7891166666666667|
| 1465727| 0.22228516853932587|
|   53634|5.053559737158626...|
|10029836|   2.968138526315789|
| 9694883|  0.6520283333333332|
| 6383217|  0.8986178082191781|
|21617467|           400.60578|
| 6232300|          0.29874625|
| 6343467|   3.685510689655173|
| 7980038|  0.7831369318181818|
|   75149|  0.7814723611111111|
| 3342932|   4.656967772727272|
|   81501|  1557559.9858973613|
+--------+--------------------+
only showing top 20 rows



## Team Dim
Metric: Total number of submissions per Team

In [4]:
agg_team = facts.groupBy("TeamKey").agg(F.count("*").alias("SubmissionCount"))
agg_team.show()


+-------+---------------+
|TeamKey|SubmissionCount|
+-------+---------------+
|7068740|             83|
| 441884|             53|
|5298640|              1|
|1171449|             19|
| 279131|             26|
|4198967|             72|
|4822531|             44|
|6821366|             79|
|2703409|              8|
|1861297|             40|
|  51595|            145|
| 577243|            204|
|  16386|              5|
|4067199|             10|
|3376035|            107|
|4561376|              1|
| 219523|            155|
|3785210|              1|
|1826992|             37|
| 412039|             52|
+-------+---------------+
only showing top 20 rows



## Competition Dim
Metric: Average private score per competition

In [5]:
agg_comp = facts.groupBy("CompetitionKey").agg(
    F.avg("PrivateScoreLeaderboardDisplay").alias("AvgPrivateScore")
)
agg_comp.show()


+--------------+---------------+
|CompetitionKey|AvgPrivateScore|
+--------------+---------------+
|          NULL|       Infinity|
+--------------+---------------+



## Time Dim
Metric: Number of submissions per year-month

In [6]:
from pyspark.sql.functions import year, month

agg_time = (
    facts.withColumn("Year", year("TimeKey"))
    .withColumn("Month", month("TimeKey"))
    .groupBy("Year", "Month")
    .agg(F.count("*").alias("SubmissionCount"))
    .orderBy("Year", "Month")
)
agg_time.show()


+----+-----+---------------+
|Year|Month|SubmissionCount|
+----+-----+---------------+
|2010|    4|              3|
|2010|    5|            186|
|2010|    6|            304|
|2010|    7|            612|
|2010|    8|           1528|
|2010|    9|           1703|
|2010|   10|           1925|
|2010|   11|           1487|
|2010|   12|           2263|
|2011|    1|           3157|
|2011|    2|           3708|
|2011|    3|           2551|
|2011|    4|           4294|
|2011|    5|           2616|
|2011|    6|           1682|
|2011|    7|           2258|
|2011|    8|           3445|
|2011|    9|           3607|
|2011|   10|           5116|
|2011|   11|           7653|
+----+-----+---------------+
only showing top 20 rows

